# 4. Word2Vec + CNN (GPU)

# 對比四種方法
| **方法**                  | **特徵工程**                | **模型**                    | **準確率預估** | **優勢**                                     | **劣勢**                                      | **GPU 支援**            |
|--------------------------|----------------------------|-----------------------------|----------------|---------------------------------------------|----------------------------------------------|-------------------------|
| **TF-IDF + 隨機森林**      | 稀疏特徵表示，詞頻與逆文檔頻率權重 | 隨機森林                    | 75%-82%       | 模型穩定性強，對噪聲和高維數據不敏感          | 無法處理非線性模式，對語義信息利用不足           | 不支持                  |
| **TF-IDF + Boosting**      | 稀疏特徵表示，詞頻與逆文檔頻率權重 | XGBoost 或 LightGBM         | 78%-85%       | 擅長處理稀疏特徵，對錯分樣本有良好適應能力      | 訓練成本略高，需調參以達到最佳效果              | 支持（顯著加速，適合大數據集）|
| **Word2Vec + 隨機森林**     | 詞嵌入，計算句向量平均值       | 隨機森林                    | 72%-80%       | 能結合詞嵌入語義特徵，提升語義捕捉能力          | 詞嵌入需預處理，隨機森林對非線性語義的處理有限     | 不支持                  |
| **Word2Vec + CNN**         | 詞嵌入，保留語序            | 卷積神經網絡                | 75%-85%       | 捕捉局部語義特徵，對短文本效果佳               | 訓練需較多資源，對長文本效果有限               | 支持（顯著加速）         |
| **BERT 嵌入 + Transformer**| 上下文語義嵌入，保留全局語義 | 預訓練 BERT 模型             | 85%-90%       | 能捕捉上下文語義，分類準確率最高               | 訓練和推理成本高，需要大量數據和資源支持         | 支持（必要，否則速度較慢） |

In [ ]:
#kaggle 前置作業
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# 引入必要的庫
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 這部分請小心執行，這是 "未做過"  雜訊處理的程式碼

In [ ]:
# 資料讀取與處理
data = []
with open('/kaggle/input/dm-2024-isa-5810-lab-2-homework/tweets_DM.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))
f.close()

# 這部分請小心執行，這是 "做過" 雜訊處理的程式碼

In [ ]:
import json
import re
import emoji
import pandas as pd

# 定義表情符號到關鍵字的映射字典
emoji_dict = {
    '😂': '[joy]',
    '❤️': '[love]',
    '😍': '[adoration]',
    '😭': '[cry]',
    '❤': '[care]',
    '😊': '[happy]',
    '🙏': '[pray]',
    '😘': '[kiss]',
    '💕': '[love_each_other]',
    '🔥': '[fire]',
    '😩': '[weary]',
    '🤔': '[think]',
    '💯': '[perfect]',
    '💙': '[loyalty]',
    '🙄': '[annoyed]',
    '😁': '[happy]',
    '🙌': '[celebrate]',
    '🙏🏾': '[pray]',
    '👍': '[approve]',
    '🙏🏽': '[pray]'
}

# 定義清理推文文本的函數
def clean_tweet(text, emoji_dict):
    # 將定義的表情符號替換為對應的關鍵詞
    for emj, keyword in emoji_dict.items():
        text = text.replace(emj, keyword)
    # 移除其餘的表情符號
    text = emoji.replace_emoji(text, replace='')
    # 移除 <LH> 標籤
    text = re.sub(r'<LH>', '', text)
    # 移除多餘的空白字元
    text = text.strip()
    return text

# 讀取推文資料
data1 = []
with open('/kaggle/input/dm-2024-isa-5810-lab-2-homework/tweets_DM.json', 'r') as f:
    for line in f:
        data1.append(json.loads(line))

# 處理每條推文並儲存結果
processed_tweets = []
for entry in data1:
    # 檢查 '_source' 和 'tweet' 是否存在於記錄中
    if '_source' in entry and 'tweet' in entry['_source']:
        tweet = entry['_source']['tweet']
        # 檢查 'text' 是否存在於 'tweet' 中
        if 'text' in tweet:
            tweet_text = tweet['text']
            cleaned_text = clean_tweet(tweet_text, emoji_dict)
            # 創建處理後的推文記錄，保留 '_source' 和 'tweet'
            processed_tweet = {
                '_source': {
                    'tweet': tweet.copy()
                }
            }
            # 更新清理後的文本
            processed_tweet['_source']['tweet']['text'] = cleaned_text
            processed_tweets.append(processed_tweet)
        else:
            print("記錄中缺少 'text' 鍵")
    else:
        print("記錄中缺少 '_source' 或 'tweet' 鍵")

# 將處理後的推文資料存儲為 JSON 檔案
with open('/kaggle/working/tweets_DM_filtered_1.json', 'w') as outfile:
    json.dump(processed_tweets, outfile, ensure_ascii=False, indent=4)



# 將處理後的資料轉換為 DataFrame
df_processed = pd.DataFrame(processed_tweets)

# 定義輸出目錄和檔案名稱
output_dir = '/kaggle/working/'
output_file = 'tweets_DM_filtered_1.json'

# 檢查並創建目錄（如果不存在）
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 將處理後的 DataFrame 儲存為 JSON 檔案
output_path = os.path.join(output_dir, output_file)
df_processed.to_json(output_path, orient='records', lines=True, force_ascii=False)

# 把 /kaggle/working/tweets_DM_filtered_1.json 載入成為 data , 可樣就可以跟原本程式合併

data = []
with open('/kaggle/working/tweets_DM_filtered_1.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))
f.close()
print("ok")

# 下面就都一樣了

In [ ]:


emotion = pd.read_csv('/kaggle/input/dm-2024-isa-5810-lab-2-homework/emotion.csv')
data_identification = pd.read_csv('/kaggle/input/dm-2024-isa-5810-lab-2-homework/data_identification.csv')

df = pd.DataFrame(data)
_source = df['_source'].apply(lambda x: x['tweet'])
df = pd.DataFrame({
    'tweet_id': _source.apply(lambda x: x['tweet_id']),
    'hashtags': _source.apply(lambda x: x['hashtags']),
    'text': _source.apply(lambda x: x['text']),
})
df = df.merge(data_identification, on='tweet_id', how='left')

train_data = df[df['identification'] == 'train']
test_data = df[df['identification'] == 'test']
train_data = train_data.merge(emotion, on='tweet_id', how='left')
train_data.drop_duplicates(subset=['text'], keep=False, inplace=True)

# 資料樣本化
train_data_sample = train_data.sample(frac=0.3, random_state=42)
y_train_data = train_data_sample['emotion']
X_train_data = train_data_sample['text']

# 標籤編碼
le = LabelEncoder()
y_train = le.fit_transform(y_train_data)

# 訓練與測試分割
X_train, X_val, y_train, y_val = train_test_split(X_train_data, y_train, test_size=0.2, random_state=42, stratify=y_train)

# 文字 Tokenization 與 Padding
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

max_len = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=max_len, padding='post')

# Word2Vec 初始化（可選：如果有預訓練詞嵌入則替換）
embedding_dim = 100
word_index = tokenizer.word_index
vocab_size = min(len(word_index) + 1, 10000)
embedding_matrix = np.random.uniform(-1, 1, (vocab_size, embedding_dim))

# 模型構建 (CNN)
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=True),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(le.classes_), activation='softmax')
])

# 模型編譯
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練
with tf.device('/gpu:0'):  # 強制使用 GPU
    history = model.fit(
        X_train_pad, y_train,
        validation_data=(X_val_pad, y_val),
        batch_size=128,
        epochs=10,
        verbose=1
    )

# 測試資料預測
test_texts = test_data['text']
test_seq = tokenizer.texts_to_sequences(test_texts)
X_test_pad = pad_sequences(test_seq, maxlen=max_len, padding='post')
y_test_pred = model.predict(X_test_pad)

# 將預測結果轉換為標籤
y_test_pred_labels = le.inverse_transform(np.argmax(y_test_pred, axis=1))

# 組裝提交檔案
submission = pd.DataFrame({
    'tweet_id': test_data['tweet_id'],
    'emotion': y_test_pred_labels
})

# submission.to_csv('/kaggle/working/submission.csv', index=False)
print("ok")

In [ ]:
submission.head()

4. Word2Vec + CNN (GPU)
# 分析部分 : 混淆矩陣 , Classification Report

## 準確率紀錄 :

## 資料沒過濾過
- Accuracy: 
0.5514

## 資料有過濾過
- Accuracy:
0.5312

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

# 1. 列出混淆矩陣
y_val_pred = np.argmax(model.predict(X_val_pad), axis=1)
conf_matrix = confusion_matrix(y_val, y_val_pred)
print("\n", "----", "\n")
print("Confusion Matrix:")
print(conf_matrix)

# 2. 畫出混淆矩陣
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.title("Confusion Matrix Heatmap")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

print("\n", "----", "\n")

# 3. 印出 Classification Report
class_report = classification_report(y_val, y_val_pred, target_names=le.classes_)
print("Classification Report:")
print(class_report)

print("\n", "----", "\n")

# 4. 印出 Accuracy
accuracy = accuracy_score(y_val, y_val_pred)
print("Accuracy:")
print(f"{accuracy:.4f}")
